In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
cols = ['action', 'step', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
        'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']
df = pd.read_csv('train.csv', header = 0, names = cols)
print('df.shape:', df.shape)
df.head()

df.shape: (2372805, 11)


,action,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,0,TRANSFER,10.00,C0198526315,30112.00,30102.00,C4653045645,21927.84,21937.84,0,0
1,1,CASH_IN,104925.19,C9864462944,42.23,104967.41,C7853342674,0.00,0.00,0,0
2,1,CASH_IN,37383.17,C1474610910,49.83,37433.01,C3584357969,0.00,0.00,0,0
3,1,CASH_IN,131908.49,C1474610910,37433.01,169341.49,C4996589500,0.00,0.00,0,0
4,1,CASH_IN,127105.65,C1474610910,169341.49,296447.14,C5989915138,0.00,0.00,0,0


In [ ]:
majority_class = df.isFraud.mode()[0]
y_pred = np.full(shape = df.isFraud.shape, fill_value = majority_class)
accuracy_score(df.isFraud, y_pred)

0.9999249833003555

In [ ]:
print(classification_report(df.isFraud, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2372627
           1       0.00      0.00      0.00       178

    accuracy                           1.00   2372805
   macro avg       0.50      0.50      0.50   2372805
weighted avg       1.00      1.00      1.00   2372805



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X = df.drop(['isFraud', 'action', 'nameOrig', 'nameDest'], axis = 1)
y = df.isFraud
rus = RandomUnderSampler(sampling_strategy=0.8)
X_res, y_res = rus.fit_resample(X, y)
print(X_res.shape, y_res.shape)
print(pd.value_counts(y_res))

(400, 7) (400,)
0    222
1    178
Name: isFraud, dtype: int64


In [ ]:
cols_numeric = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
                'oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud']
df_rus = pd.DataFrame(X_res, columns = cols_numeric)
df_rus.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,CASH_IN,82263.53,2365384.48,2447648.01,27147.24,27147.24,0
1,PAYMENT,3368.08,2716128.74,2712760.66,53068.33,56436.40,0
2,PAYMENT,7005.70,2124786.17,2117780.47,104259.50,111265.20,0
3,CASH_IN,3727.76,3833773.97,3837501.74,30382.29,30382.29,0
4,PAYMENT,19858.54,2648102.34,2628243.80,90173.12,110031.67,0


In [ ]:
from sklearn.model_selection import train_test_split
def train_validation_test_split(
    X, y, train_size=0.8, val_size=0.1, test_size=0.1, 
    random_state=None, shuffle=True):
    assert int(train_size + val_size + test_size + 1e-7) == 1
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=shuffle)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val,    test_size=val_size/(train_size+val_size), 
        random_state=random_state, shuffle=shuffle)
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = train_validation_test_split(
    X_res, y_res, train_size=0.8, val_size=0.1, test_size=0.1, random_state=1)

class_weight = {0: 4, 1: 5}
model = LogisticRegression(class_weight=class_weight)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print('accuracy', accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92        18
           1       0.95      0.91      0.93        22

    accuracy                           0.93        40
   macro avg       0.92      0.93      0.92        40
weighted avg       0.93      0.93      0.93        40

accuracy 0.925


In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy', accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        18

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Accuracy 1.0


In [ ]:
y_pred = model.predict(X)
print(classification_report(y, y_pred))
print('Accuracy:', accuracy_score(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99   2372627
           1       0.00      0.97      0.01       178

    accuracy                           0.99   2372805
   macro avg       0.50      0.98      0.50   2372805
weighted avg       1.00      0.99      0.99   2372805

Accuracy: 0.9852267674756249
